In [3]:
# Import python libraries
import psycopg2
import pandas as pd

# Import postgres server info key
from postgres_info import user, password


# Use a 'try' statement to attempt to establish a connection to the PostgreSQL database 
try:

    # Connect to the database using login info stored in 'postgres_info.py'
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="ev_db"
    )

# Use 'except' statement to handle errors connenting to the database
except psycopg2.Error as e:
    print("Error connecting to the 'ev_db' database:")
    print(e)

# Use 'else' statement to continue program if connection is successful
else:
    print("Connection to 'ev_db' database established successfully")
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # Create a variable to hold all table names and column names in our database
    select_table_names = """SELECT table_name,column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_schema = 'public';"""
    
    # Execute the SQL query above
    cursor.execute(select_table_names)
    
    # Fetch all data from the query execution above
    tables_rows = cursor.fetchall()
    
    # Assemble the data into a dataframe
    tables_rows_df = pd.DataFrame(tables_rows).sort_values(by=0)
    
    # Show the dataframe
    print(f'Available Tables and Columns:  {tables_rows_df}')

    # Close cursor 
    cursor.close()

    # Close the connection to the database
    conn.close()


# Create a 'while True:' statement to allow user to run script again until they quit
while True:


    # Create a 'try' statement to attempt to reconnect to database
    try:

        # Reconnect to the database
        conn = psycopg2.connect(
            host="127.0.0.1",
            port="5432",
            user=user,
            password=password,
            database="ev_db"
        )
    
    # Use 'except' statement to handle errors connenting to the database
    except psycopg2.Error as e:
        print("Error reconnecting to the 'ev_db' database:")
        print(e)
        # Break 'while True:' loop 
        break

    # Use 'else' statement to continue program if reconnection is successful
    else:
        print("Reconnection to 'ev_db' database established successfully")


    # Create another 'try' statement to run user selected SQL query and handle errors that occuer
    try:
        
        # Recreate the cursor
        cursor = conn.cursor()

        # Create an input variable for users to write their own SQL query
        input_query = input("Enter a SQL query to download table as CSV: ")

        # Execute the query that holds the user selection
        cursor.execute(input_query)
        
        # Print statement to show user's query executed successfully 
        print(f'Query Executed Successfully: {input_query}')
        
        # Fetch all rows from the result
        rows = cursor.fetchall()
        
        # Use cursor.description to get column names from SQL query
        column_names = [desc[0] for desc in cursor.description]
        
        # Assemble data into a dataframe
        input_df = pd.DataFrame(rows, columns=column_names)
        
        # Ask for name for CSV
        csv_name = input("Enter a name for the CSV: ")
        
        # Save as csv to Input_App_CSVs directory
        input_df.to_csv(f"Input_App_CSVs/{csv_name}.csv", index=False)
        
        # Print statement to show user CSV was saved successfully
        print(f"Data successfully saved to 'Input_App_CSVs' as '{csv_name}'")
        
        # Close the cursor
        cursor.close()

        # Show dataframe
        print(f'Results of {input_query}')
        print(input_df)

    # Use expect to handle error    
    except psycopg2.Error as e:
        print(f"Error: {e}")


    # Close connection
    conn.close()

    # Use an input variable to ask if the user if they want to run the program again
    run_again = input("Do you want to run the script again? (yes/no): ").strip().lower()

    # Create an 'if' statement to handle user typing anything but 'yes'
    if run_again != "yes":
        print("Thank you for using our app!")
        # Close connection
        conn.close()
        # Break the program if user does not type 'yes'
        break


# Show dataframe
print('')
print(f'Results of {input_query}')
input_df

Connection to 'ev_db' database established successfully
Available Tables and Columns:                      0                     1
23   cafv_eligibility               cafv_id
25   cafv_eligibility      cafv_eligibility
3       county_income      percapita_income
24      county_income             fips_code
26      county_income           income_year
29      location_info                county
1       location_info              latitude
2       location_info             longitude
27      location_info           postal_code
5       location_info                  city
6       location_info                 state
7       location_info  legislative_district
8       location_info     census_tract_2020
28      location_info             fips_code
21  utility_companies    utility_company_id
22  utility_companies  utility_company_name
12      vehicle_types                 model
11      vehicle_types                  make
9       vehicle_types       vehicle_type_id
13      vehicle_types            

,dol_vehicle_id,vehicle_type_id,postal_code,fips_code,utility_company_id,cafv_id,vin,electric_range,base_msrp
0,125450447,vm0027,98177,53033,uc001,cafv1,xxxxxxCP8D,75.0,0.00
1,101662900,vm0199,98112,53033,uc001,cafv1,xxxxxxE45K,270.0,0.00
2,272118717,vm0288,98359,53035,uc002,cafv2,xxxxxxE28M,0.0,0.00
3,349372929,vm0129,98501,53067,uc002,cafv3,xxxxxxFP6H,25.0,0.00
4,171625653,vm0081,98506,53067,uc002,cafv3,xxxxxxCU9G,19.0,0.00
...,...,...,...,...,...,...,...,...,...
216735,122822822,vm0021,98802,53017,uc025,cafv1,xxxxxxE44D,38.0,0.00
216736,267143887,vm0479,98229,53073,uc023,cafv1,xxxxxxLF9R,33.0,0.00
216737,274988388,vm0518,98052,53033,uc004,cafv2,xxxxxxEE9R,0.0,0.00
216738,117353064,vm0021,98329,53053,uc022,cafv1,xxxxxxE49D,38.0,0.00
